# 680 Project 2: Air Quality Prediction
Name: Joi Chu-Ketterer <br>  Date: 5/1/2020 <br>Course: DSC680 - Applied Data Science

In [4]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt

# Data Preparation

This section cleans and prepares the raw data for analysis. There are three datasets in total. The first two are trace gas levels, and the third is COVID19 counts for Southern California. 

_Importing Data_

In [11]:
o3_1 = pd.read_csv('o3_41619-101620.csv')
o3_2 = pd.read_csv('o3_101719-41720.csv')
o3_3 = pd.read_csv('o3_41819-43020.csv') 

In [12]:
no2_1 = pd.read_csv('no2_41619-101620.csv')
no2_2 = pd.read_csv('no2__101719-41720.csv')
no2_3 = pd.read_csv('no2__41819-43020.csv') 

In [13]:
o3_combine = o3_1.append(o3_2, ignore_index = True)
o3_df = o3_combine.append(o3_3, ignore_index = True)
o3_df.head()

,Date Time,Site Name,Value,Averaging Period,Units,California State Standard,National Ambient Air Quality Standard by U.S. EPA
0,4/16/19 0:00,Central Los Angeles,3.3,1-hr,pphm,9,NaN
1,4/16/19 1:00,Central Los Angeles,3.1,1-hr,pphm,9,NaN
2,4/16/19 2:00,Central Los Angeles,3.1,1-hr,pphm,9,NaN
3,4/16/19 3:00,Central Los Angeles,3.0,1-hr,pphm,9,NaN
4,4/16/19 4:00,Central Los Angeles,3.1,1-hr,pphm,9,NaN


In [14]:
no2_combine = no2_1.append(no2_2, ignore_index = True)
no2_df = no2_combine.append(no2_3, ignore_index = True)
no2_df.head()

,Date Time,Site Name,Value,Averaging Period,Units,California State Standard,National Ambient Air Quality Standard by U.S. EPA
0,4/16/19 0:00,Central Los Angeles,0.8,1-hr,pphm,18,NaN
1,4/16/19 1:00,Central Los Angeles,0.9,1-hr,pphm,18,NaN
2,4/16/19 2:00,Central Los Angeles,0.8,1-hr,pphm,18,NaN
3,4/16/19 3:00,Central Los Angeles,1.0,1-hr,pphm,18,NaN
4,4/16/19 4:00,Central Los Angeles,0.9,1-hr,pphm,18,NaN


In [15]:
covid = pd.read_csv('county_totals.csv')
covid.head()

,date,county,fips,confirmed_cases,deaths,new_confirmed_cases,new_deaths
0,2020-01-26,Alameda,1,0,0,NaN,NaN
1,2020-01-31,Alameda,1,0,0,0.0,0.0
2,2020-02-02,Alameda,1,0,0,0.0,0.0
3,2020-02-20,Alameda,1,0,0,0.0,0.0
4,2020-02-21,Alameda,1,0,0,0.0,0.0


In [16]:
no2_df = no2_df.drop(columns=['Site Name', 'Averaging Period', 'Units', 'California State Standard', 'National Ambient Air Quality Standard by U.S. EPA'])
o3_df = o3_df.drop(columns=['Site Name', 'Averaging Period', 'Units', 'California State Standard', 'National Ambient Air Quality Standard by U.S. EPA'])

no2_df = no2_df.rename(columns={"Date Time": "DATETIME", "Value": "NO2"})
o3_df = o3_df.rename(columns={"Date Time": "DATETIME", "Value": "VALUE"})

In [17]:
no2_df.head()

,DATETIME,NO2
0,4/16/19 0:00,0.8
1,4/16/19 1:00,0.9
2,4/16/19 2:00,0.8
3,4/16/19 3:00,1.0
4,4/16/19 4:00,0.9


In [18]:
o3_df.head()

,DATETIME,VALUE
0,4/16/19 0:00,3.3
1,4/16/19 1:00,3.1
2,4/16/19 2:00,3.1
3,4/16/19 3:00,3.0
4,4/16/19 4:00,3.1


_Trace Gas Data Cleaning_

In [19]:
no2_df['O3'] = o3_df['VALUE']
no2_df['DATETIME'] = pd.to_datetime(no2_df['DATETIME'])
no2_df.head()

,DATETIME,NO2,O3
0,2019-04-16 00:00:00,0.8,3.3
1,2019-04-16 01:00:00,0.9,3.1
2,2019-04-16 02:00:00,0.8,3.1
3,2019-04-16 03:00:00,1.0,3.0
4,2019-04-16 04:00:00,0.9,3.1


_COVID19 Count Data Cleaning_

In [20]:
covid = covid.drop(columns=['fips', 'deaths','new_confirmed_cases', 'new_deaths'])
covid['date'] = pd.to_datetime(covid['date'])
covid_la = covid[(covid['county'] == "Los Angeles")]
covid_la.head()

,date,county,confirmed_cases
795,2020-01-26,Los Angeles,1
796,2020-01-27,Los Angeles,1
797,2020-01-28,Los Angeles,1
798,2020-01-29,Los Angeles,1
799,2020-01-30,Los Angeles,1


_Combining Datasets_

In [21]:
combined = pd.merge(left=no2_df, right=covid_la, how='left', left_on='DATETIME', right_on='date')
combined.head()

,DATETIME,NO2,O3,date,county,confirmed_cases
0,2019-04-16 00:00:00,0.8,3.3,NaT,NaN,NaN
1,2019-04-16 01:00:00,0.9,3.1,NaT,NaN,NaN
2,2019-04-16 02:00:00,0.8,3.1,NaT,NaN,NaN
3,2019-04-16 03:00:00,1.0,3.0,NaT,NaN,NaN
4,2019-04-16 04:00:00,0.9,3.1,NaT,NaN,NaN


In [23]:
combined.tail(30)

,DATETIME,NO2,O3,date,county,confirmed_cases
8623,2020-04-29 13:00:00,1.7,2.4,NaT,NaN,NaN
8624,2020-04-29 14:00:00,0.9,2.2,NaT,NaN,NaN
8625,2020-04-29 15:00:00,0.7,2.6,NaT,NaN,NaN
8626,2020-04-29 16:00:00,0.5,2.3,NaT,NaN,NaN
8627,2020-04-29 17:00:00,0.5,2.6,NaT,NaN,NaN
8628,2020-04-29 18:00:00,0.7,2.4,NaT,NaN,NaN
8629,2020-04-29 19:00:00,0.6,1.8,NaT,NaN,NaN
8630,2020-04-29 20:00:00,0.7,1.3,NaT,NaN,NaN
8631,2020-04-29 21:00:00,0.7,1.3,NaT,NaN,NaN
8632,2020-04-29 22:00:00,1.0,1.4,NaT,NaN,NaN


In [17]:
combined.to_csv('clean_trace.csv', index = False)